In [18]:
#imports
from tensorflow import keras
import numpy as np
import pyaudio
import time
import librosa
import os
from glob import glob
import scipy.cluster.hierarchy as hcluster
import matplotlib.pyplot as plt

In [19]:
#get speaker list

speakers = ["No Speaker"]
numClips = 23

for i in range(numClips):
    speakers.append(str(i+1))
    
print(speakers)

['No Speaker', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


In [20]:
#load model
model = keras.models.load_model('Trained Model')

In [21]:
def getFeatures(audio_Paths):
    
    data_X = []
    
    for path in audio_Paths:
        
        audioLength = librosa.get_duration(filename=path)
        
        if audioLength != 1.0:
            continue
        
        audioFeatureArray = []        
        y, sr = librosa.load(path)
        
        #mfcc
        mfccArray = librosa.feature.mfcc(y=y, sr=sr)
        audioFeatureArray.append(mfccArray.flatten())
        audioFeatureNumpyArray = np.array(audioFeatureArray)
        
        #zero_crossing_rate
        zeroCrossingArray = librosa.feature.zero_crossing_rate(y=y)
        np.append(audioFeatureNumpyArray, zeroCrossingArray.flatten())
        
        #spectral_rolloff
        spectralRollOffArray = librosa.feature.spectral_rolloff(y=y, sr=sr)
        np.append(audioFeatureNumpyArray, spectralRollOffArray.flatten())
        
        data_X.append(audioFeatureNumpyArray.flatten())
        
    return data_X

In [22]:
def findAllAudioFilePaths(speaker):
    audioFilesPaths = [y for x in os.walk("Dataset/Youtube Speech Dataset/Dataset/{}".format(speaker)) for y in glob(os.path.join(x[0], '*.wav'))]
    return audioFilesPaths

In [23]:
#For confusion matrix
def runModel(speaker):
   
    audio_Paths = findAllAudioFilePaths(speaker)

    audioFeatureArray = getFeatures(audio_Paths) 
            
    validation_x = np.array(audioFeatureArray)

    predictScore = model.predict(validation_x)        
    classes = np.argmax(predictScore, axis = 1)

    speakerCount = dict()
    for classPredict in classes:
        speakerCount[classPredict] = speakerCount.get(classPredict, 0) + 1


    print("")
    print("-------------------------")
    print("Testing model for {}".format(speaker))
    
    for speaker in speakerCount: 
        print("Speaker: {} ----> Votes: {}".format(speakers[int(speaker)], speakerCount[speaker] )) 
    print("-------------------------")
    print("")

In [24]:
#for speaker in speakers:
#    runModel(speaker)

In [25]:
#runModel("1")


-------------------------
Testing model for 1
Speaker: 1 ----> Votes: 118
Speaker: 19 ----> Votes: 1
Speaker: 20 ----> Votes: 2
Speaker: 22 ----> Votes: 1
Speaker: 6 ----> Votes: 1
Speaker: No Speaker ----> Votes: 1
-------------------------

